<a href="https://colab.research.google.com/github/Strugoeden121/ATAM1/blob/main/Major_HW1_314884602_324490135.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from matplotlib import pylab
params = {'xtick.labelsize': 18,
 'ytick.labelsize': 18,
 'axes.titlesize' : 22,
 'axes.labelsize' : 20,
 'legend.fontsize': 18,
 'legend.title_fontsize': 22,
 'figure.titlesize': 24 }
pylab.rcParams.update(params)

In [2]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('virus_data.csv', header=0)

In [16]:
df.shape


(1250, 26)

In [15]:
df['conversations_per_day'].value_counts()

3     224
2     215
4     190
5     156
6     111
1     104
8      72
7      60
9      39
10     23
11     19
12     12
13      9
14      6
17      4
15      2
16      2
19      1
22      1
Name: conversations_per_day, dtype: int64